# Imports

In [ ]:
!pip install -U tensorflow==2.6.4
!pip install cloud-tpu-client

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pydicom
import os
import cv2
import tensorflow as tf
from cloud_tpu_client import Client
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras import backend as K
from google.cloud import storage

In [ ]:
tf.version.VERSION

In [ ]:
from cloud_tpu_client import Client
Client().configure_tpu_version(tf.__version__, restart_type='ifNeeded')

# Define GCS Storage Area

In [ ]:
client_area = 'kagglersna01'
storage_client = storage.Client(project=client_area)

# Delete All

In [ ]:
def delete_file(filepath):
    print('deleting ' + filepath + ' from local')
    os.remove('/kaggle/working/' + filepath)
    
def clear_all_local():
    for k in os.listdir('/kaggle/working/'):
        if k == '.virtual_documents':
            continue
        delete_file(k)

In [ ]:
import shutil

In [ ]:
def clear_all():
    shutil.rmtree('/kaggle/working/checkpoints')
    clear_all_local()
#clear_all()

# Init TPU

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Setup Params

In [ ]:
imsize = [512, 512, 3] 
epoch = 20
batchsize = 16 * strategy.num_replicas_in_sync

trim = 43764 
validim = 10942 
step_per_epoch = (trim // batchsize) * 5
steps_per_val_epoch = (validim // batchsize) + 1

# Parsing Function

In [ ]:
def parse_example(tfrecord):
    feature_desc = {
        'image': tf.io.FixedLenFeature([], tf.string, default_value=""),
        'label': tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        'age': tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        'impant': tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        'laterality': tf.io.FixedLenFeature([], tf.string, default_value=""),
        'view': tf.io.FixedLenFeature([], tf.string, default_value=""),
        'diff_neg': tf.io.FixedLenFeature([], tf.int64, default_value=-1)
    }
    
    example = tf.io.parse_single_example(tfrecord, feature_desc)
    image = tf.io.parse_tensor(example["image"], out_type=tf.uint8)
    image = tf.reshape(image, shape=imsize)
    return image, tf.cast(example["label"], tf.float32)

# Data Augmentations

> **INPUT** - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3] 

> **OUTPUT** - image with CT squares of side size SZ*DIM removed

In [ ]:
def dropout(image, DIM=512, PROBABILITY=0.6, CT=5, SZ=0.10):
    
    # DO DROPOUT WITH PROBABILITY DEFINED ABOVE
    P = tf.cast( tf.random.uniform([], 0, 1)<PROBABILITY, tf.int32)
    if (P==0) | (CT==0) | (SZ==0): 
        return image
    
    for k in range(CT):
        
        # CHOOSE RANDOM LOCATION
        x = tf.cast(tf.random.uniform([], 0, DIM),tf.int32)
        y = tf.cast(tf.random.uniform([], 0, DIM),tf.int32)
        
        # COMPUTE SQUARE 
        WIDTH = tf.cast(SZ*DIM, tf.int32) * P
        ya = tf.math.maximum(0, y-WIDTH//2)
        yb = tf.math.minimum(DIM, y+WIDTH//2)
        xa = tf.math.maximum(0, x-WIDTH//2)
        xb = tf.math.minimum(DIM, x+WIDTH//2)
        
        # DROPOUT IMAGE
        one = image[ya:yb, 0:xa, :]
        two = tf.zeros([yb-ya, xb-xa, 3], dtype=image.dtype) 
        three = image[ya:yb, xb:DIM, :]
        middle = tf.concat([one, two, three], axis=1)
        image = tf.concat([image[0:ya,:,:], middle, image[yb:DIM, :, :]], axis=0)
        image = tf.reshape(image, [DIM, DIM, 3])

    return image

In [ ]:
data_augm_lay = keras.Sequential(
    [
        keras.layers.RandomZoom(0.3)
    ]
)

In [ ]:
def augment_image(image, y):
    
    image = tf.image.random_brightness(image, 0.10)
    image = tf.image.random_contrast(image, 0.90, 1.40)
    image = tf.image.random_saturation(image, 0.50, 2.00)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    image = data_augm_lay(image)
    image = dropout(image)
    
    return image, y

In [ ]:
sample_per = 1 - (2.0770496298327394 / 10)

In [ ]:
def undersample_majority(x, y):
    return y == 1 or tf.random.uniform([]) > sample_per

# Get Dataset From GCS Files

In [ ]:
def record_dataset(filepaths, shuffle_buffer_size=5000, batch_size=32, training=True, ordered=False):
    
    ignore_order = tf.data.Options()
    
    if not ordered:
        ignore_order.experimental_deterministic = False
    
    dataset = tf.data.TFRecordDataset(filepaths, num_parallel_reads=tf.data.AUTOTUNE)
    
    # dataset = dataset.cache()
    
    dataset = dataset.map(parse_example, num_parallel_calls=tf.data.AUTOTUNE)
    
    if training:
        dataset = dataset.filter(undersample_majority)
        dataset = dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.with_options(ignore_order)
        dataset = dataset.shuffle(shuffle_buffer_size)
        dataset = dataset.repeat()
        
    dataset = dataset.batch(batchsize)
    
    dataset = dataset.prefetch(1)
    return dataset

In [ ]:
trainpaths = ['gs://' + 'train_batches' + '/' + "{}_batch_{:0>3}.tfrecord".format('train', index) for index in range(0, 200)]
validpaths = ['gs://' + 'train_batches' + '/' + "{}_batch_{:0>3}.tfrecord".format('valid', index) for index in range(0, 100)]

train_set = record_dataset(trainpaths, batchsize)
valid_set = record_dataset(validpaths, batchsize, training=False)

# Metric/s

In [ ]:
class pFBeta(tf.keras.metrics.Metric):
    """Compute overall probabilistic F-beta score."""
    def __init__(self, beta=1, epsilon=1e-5, name='pF1', **kwargs):
        super().__init__(name=name, **kwargs)
        self.beta = beta
        self.epsilon = epsilon
        self.pos = self.add_weight(name='pos', initializer='zeros')
        self.ctp = self.add_weight(name='ctp', initializer='zeros')
        self.cfp = self.add_weight(name='cfp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 0, 1)
        pos = tf.reduce_sum(y_true)
        ctp = tf.reduce_sum(y_pred[y_true==1])
        cfp = tf.reduce_sum(y_pred[y_true==0])
        self.pos.assign_add(pos)
        self.ctp.assign_add(ctp)
        self.cfp.assign_add(cfp)

    def result(self):
        beta_squared = self.beta * self.beta
        c_precision = self.ctp / (self.ctp + self.cfp + self.epsilon)
        c_recall = self.ctp / (self.pos + self.epsilon)
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return tf.cond(c_precision > 0 and c_recall > 0, lambda: result, lambda: 0.0)

> Find Best Threshold

In [ ]:
def pfbeta_tf(labels, preds, beta=1):
    eps = 1e-5
    preds = tf.clip_by_value(preds, 0, 1)
    y_true_count = tf.reduce_sum(labels)
    ctp = tf.reduce_sum(preds[labels==1])
    cfp = tf.reduce_sum(preds[labels==0])
    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp + eps)
    c_recall = ctp / (y_true_count + eps)
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + eps)
        return result
    else:
        return tf.constant(0.0, dtype=tf.float32)

# finds best pf1 using thresholds
def pfbeta_thr(labels, preds):
    thrs = tf.range(0, 1, 0.05)
    best_score = tf.constant(0, dtype=tf.float32)
    for thr in thrs:
        score = pfbeta_tf(labels, tf.cast(preds>thr, tf.float32))
        best_score = tf.cond(score > best_score, lambda: score, lambda: best_score)
    return best_score

pfbeta_thr.__name__='pF1_thr'

# Build Model

In [ ]:
class IdentityUnit(keras.layers.Layer):
    def __init__(self, filters, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.filters = filters
        
        filter_a, filter_b = self.filters
        
        self.main_layers = [
            keras.layers.Conv2D(filter_a, 1, strides=1, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filter_a, 3, strides=1, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filter_b, 1, strides=1, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
        ]
            
    def call(self, inputs):
        z = inputs
        for layer in self.main_layers:
            z = layer(z)
        return self.activation(z + inputs)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "filters": self.filters,
                "activation": keras.activations.serialize(self.activation)}

In [ ]:
class ConvUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.filters = filters
        self.strides = strides
        
        filter_a, filter_b = self.filters
        
        self.main_layers = [
            keras.layers.Conv2D(filter_a, 1, strides=1, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filter_a, 3, strides=self.strides, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filter_b, 1, strides=1, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
        ]
        
        self.skip_layers = [
            keras.layers.Conv2D(filter_b, 1, strides=self.strides, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
        ]
            
    def call(self, inputs):
        z = inputs
        for layer in self.main_layers:
            z = layer(z)
        skip_z = inputs
        for layer in self.skip_layers:
            skip_z = layer(skip_z)
        return self.activation(z + skip_z)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "filters": self.filters,
                "strides": self.strides,
                "activation": keras.activations.serialize(self.activation)}

In [ ]:
def norm(image):
    image = tf.cast(image, tf.float32)
    image = tf.image.per_image_standardization(image)
    return image

In [ ]:
def set_model_params():
    metrics = [
        pFBeta(name='pF1'),
        pfbeta_thr,
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.BinaryAccuracy()
    ]

    loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    return metrics, loss, optimizer

In [ ]:
print(os.listdir('./model/'))

In [ ]:
from_saved = True
model_name = 'resnet101-V3.h5'

In [ ]:
if from_saved:    
    with strategy.scope():
        load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
        model = tf.keras.models.load_model(
            './model/' + model_name, 
            options=load_locally, 
            custom_objects={
                "ConvUnit": ConvUnit, 
                "IdentityUnit": IdentityUnit, 
                'pFBeta': pFBeta,
                'pF1_thr': pfbeta_thr
            }
        )
        
else:
    with strategy.scope():
        model = keras.models.Sequential()
        model.add(keras.layers.InputLayer(input_shape=imsize))
        model.add(keras.layers.Lambda(lambda x: norm(x)))
        model.add(keras.layers.Conv2D(64, 7, strides=2, padding='same', use_bias=False))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))

        for filters in [[(64, 256)] * 3] + [[(128, 512)] * 4] + [[(256, 1024)] * 23] + [[(512, 2048)] * 3]:
            f = filters[0]
            model.add(ConvUnit(f, strides=2))
            for f in filters[1:]:
                model.add(IdentityUnit(f))

        model.add(keras.layers.GlobalAvgPool2D())
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(512, activation="relu"))
        model.add(keras.layers.Dropout(0.30))
        model.add(keras.layers.Dense(1, activation='sigmoid'))

        metrics = [
            pFBeta(name='pF1'),
            pfbeta_thr,
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.BinaryAccuracy()
        ]

        loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
        optimizer = keras.optimizers.Adam(learning_rate=0.001)
    
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

In [ ]:
tf.keras.backend.clear_session()

tf.config.optimizer.set_jit(True)

model.summary()

# Train

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    train_set, 
    steps_per_epoch=step_per_epoch,
    epochs=1, 
    validation_data=valid_set,
    validation_steps=steps_per_val_epoch,
    class_weight = {
        0: 1.0,
        1: 10.0
    },
    callbacks=[]
)

In [ ]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
# model.save('./model/resnet101-V3.h5', options=save_locally)